In [16]:
import numpy as np
import pandas as pd
import glob
import os
import sys
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'sets'

In [5]:
city = 'Austin'
# year = 2012
# store = pd.HDFStore(os.path.expanduser("../metadata/city_year/hourly/Austin-2012.h5"), 'r')

In [12]:
store = {}
for i in [2012, 2013, 2014, 2015, 2016, 2017]:
    store[i] = pd.HDFStore(os.path.expanduser("../metadata/city_year/hourly/Austin-{}.h5").format(i), 'r')

In [40]:
metadata_df = pd.read_csv("../metadata/metadata.csv",index_col=0)
feeds = {'use':'aggregate',
        'air1':'hvac',
         'range1':'range',
         'clotheswasher1':'wm',
         'dishwasher1':'dw',
         'microwave1':'mw',
         'lights_plugs1':'light',
         'refrigerator1':'fridge',
         'oven1':'oven',
        'car1':'ec',
         'waterheater1':'wh',
         'drye1':'dr'
        }

In [15]:
home_id = {}
for i in [2012, 2013, 2014, 2015, 2016, 2017]:
    print(i, len(store[i].keys()))
    home_id[i] = store[i].keys()

2012 207
2013 386
2014 548
2015 514
2016 368
2017 330


In [31]:
common_home_id = set(home_id[2012])
for i in [2012, 2013, 2014, 2015, 2016, 2017]:
    common_home_id = common_home_id.intersection(set(home_id[i]))
list(common_home_id)[0]

'/2158'

In [54]:
store[2012][list(common_home_id)[0]]['use']

localhour
2012-08-25 00:00:00-05:00    5473.649902
2012-08-25 01:00:00-05:00    3320.366699
2012-08-25 02:00:00-05:00    2551.583252
2012-08-25 03:00:00-05:00    2195.383301
2012-08-25 04:00:00-05:00     796.099976
2012-08-25 05:00:00-05:00    1547.533325
2012-08-25 06:00:00-05:00    1177.583374
2012-08-25 07:00:00-05:00    1203.016724
2012-08-25 08:00:00-05:00    1823.833374
2012-08-25 09:00:00-05:00    1876.050049
2012-08-25 10:00:00-05:00    2608.266602
2012-08-25 11:00:00-05:00    1587.150024
2012-08-25 12:00:00-05:00    1679.383301
2012-08-25 13:00:00-05:00     729.483337
2012-08-25 14:00:00-05:00    1573.816650
2012-08-25 15:00:00-05:00    1418.816650
2012-08-25 16:00:00-05:00    2512.983398
2012-08-25 17:00:00-05:00    3260.966553
2012-08-25 18:00:00-05:00    3260.483398
2012-08-25 19:00:00-05:00    3419.616699
2012-08-25 20:00:00-05:00    3339.949951
2012-08-25 21:00:00-05:00    3327.766602
2012-08-25 22:00:00-05:00    3770.433350
2012-08-25 23:00:00-05:00    5613.600098
2012-0

In [46]:
APPLIANCES_ORDER = ['aggregate', 'hvac', 'fridge', 'mw', 'dw', 'wm', 'oven','wh','dr','range', 'light']

In [75]:
len(store[2015]['5275'].resample('1H'))

8760

In [92]:
out = {}
homes = {}
for i in [2012, 2013, 2014, 2015, 2016, 2017]:
    out[i] = []
    homes[i] = []
    for home in store[i].keys():
        raw_data_df = store[i][home]
        data_df = raw_data_df.rename(columns=feeds)
        data_df = data_df.resample('1H')
        o =  [None] * len(APPLIANCES_ORDER)
        # Only consider homes that have data from Jan 1 to Dec 31
        day_of_year = data_df.index.dayofyear
        if day_of_year[0]==1 and day_of_year[-1]==365:
            if len(data_df)<((365*24)):
                continue
            data_df = data_df.rename(columns=feeds)
            o =  [None] * len(APPLIANCES_ORDER)
            for a_num, appliance in enumerate(APPLIANCES_ORDER):
                if appliance in data_df.columns:
                    o[a_num] = data_df[appliance].values.reshape(365, 24)
                else:
                    o[a_num] = np.zeros((365, 24))
                    o[a_num].fill(np.nan)
            out[i].append(o)
            homes[i].append(int(home[1:]))
                

In [97]:
for i in [2012, 2013, 2014, 2015, 2016, 2017]:
    print(i, len(homes[i]))

2012 0
2013 189
2014 329
2015 318
2016 0
2017 210


In [33]:
homes = {}
#for freq in ['2H','4H','12H','1H']:
for freq in ['1H']:
    print(freq)
    freq_num = int(freq[:-1])
    print(freq_num)
#     out = []
#     homes[freq] = []
#     city_data = metadata_df[metadata_df['city'] == city]
#     city_homes = city_data.index.values.astype('int')
#     for home in city_homes[:]:

#         try:
#             data_df = st['/%d' %home]['%s' %year]

#             # Only consider homes that have data from Jan 1 to Dec 31
#             day_of_year = data_df.index.dayofyear
#             if day_of_year[0]==1 and day_of_year[-1]==365:
#                 downsampled_data = data_df.resample(freq).mean()
#                 if len(downsampled_data)<((365*24)//freq_num):
#                     continue
#                 downsampled_data = downsampled_data.rename(columns=feeds)
#                 o =  [None] * len(APPLIANCES_ORDER)
#                 for a_num, appliance in enumerate(APPLIANCES_ORDER):
#                     if appliance in downsampled_data.columns:
#                         o[a_num] = downsampled_data[appliance].values.reshape(365, 24//freq_num)
#                     else:
#                         o[a_num] = np.zeros((365, 24//freq_num))
#                         o[a_num].fill(np.nan)
#                 out.append(o)
#                 homes[freq].append(home)

#             else:
#                 continue
#         except Exception as e:
#             print(e)
#     out = np.array(out)
    

   

1H
1
